In [1]:
import os
os.environ["PATH"] = "/home/jovyan/.mlspace/envs/deepseek_iana/bin:" + os.environ["PATH"]
os.environ["CUDA_HOME"] = "/home/jovyan/shares/SR008.fs2/iana_kulichenko/cuda-12.4"
os.environ["PATH"] = os.path.join(os.environ["CUDA_HOME"], "bin") + ":" + os.environ["PATH"]
os.environ["TOKENIZERS_PARALLELISM"] = "false"
print("CUDA_HOME =", os.environ["CUDA_HOME"])
print("PATH =", os.environ["PATH"])
!nvcc --version

CUDA_HOME = /home/jovyan/shares/SR008.fs2/iana_kulichenko/cuda-12.4
PATH = /home/jovyan/shares/SR008.fs2/iana_kulichenko/cuda-12.4/bin:/home/jovyan/.mlspace/envs/deepseek_iana/bin:/home/jovyan/.mlspace/envs/deepseek_iana/bin:/home/user/conda/condabin:/home/user/conda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/home/user/conda/bin
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Tue_Feb_27_16:19:38_PST_2024
Cuda compilation tools, release 12.4, V12.4.99
Build cuda_12.4.r12.4/compiler.33961263_0


In [2]:
# from transformers import Qwen2_5_VLForConditionalGeneration, AutoTokenizer, AutoProcessor
# from qwen_vl_utils import process_vision_info

# model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
#     "Qwen/Qwen2.5-VL-7B-Instruct", torch_dtype="auto", device_map="auto"
# )

In [8]:
!CUDA_VISIBLE_DEVICES=0,1,2,3,4,5,6,7

In [6]:
import os
import torch
from PIL import Image
from transformers import Qwen2_5_VLForConditionalGeneration, AutoProcessor
from qwen_vl_utils import process_vision_info

image_dir = "/home/jovyan/shares/SR008.fs2/iana_kulichenko/Experiments/train_GRPO/images_2"
eval_dir = "/home/jovyan/shares/SR008.fs2/iana_kulichenko/Experiments/train_GRPO/evaluations"
os.makedirs(eval_dir, exist_ok=True)

image_files = [
    os.path.join(image_dir, f)
    for f in os.listdir(image_dir)
    if f.lower().endswith((".jpg", ".jpeg", ".png"))
]
if not image_files:
    raise ValueError("No image files found in the specified folder.")
img_path = image_files[0]

image = Image.open(img_path).convert("RGB")
model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2.5-VL-7B-Instruct",
    torch_dtype="auto",
    device_map="auto"
)
processor = AutoProcessor.from_pretrained("Qwen/Qwen2.5-VL-7B-Instruct", use_fast=True)
evaluation_instruction = (
    "As a professional interior designer, evaluate the following 2D layout visually. "
    "Provide exactly four ratings (each a number between 0 and 1) based on:\n\n"
    "- Ergonomic & Functional Placements: Comfort, accessibility, and practicality.\n"
    "- Visual Harmony and Aesthetics: Balance and overall appeal.\n"
    "- Proportionality and Scale: Correct sizes, rotations, and proportions.\n"
    "- Readiness to Pay: Value assessment (1 = willing to purchase, 0 = not).\n\n"
    "Also, list up to three one-sentence suggestions for coordinate adjustments under 'Suggestions'. "
    "Do not propose adding or removing objects; only recommend repositioning the existing items.\n\n"
    "Output exactly as:\n"
    "### Ratings:\n"
    "- Ergonomic & Functional Placements: <rating>\n"
    "- Visual Harmony and Aesthetics: <rating>\n"
    "- Proportionality and Scale: <rating>\n"
    "- Readiness to Pay: <rating>\n"
    "### Suggestions:\n"
    "- <suggestion 1>\n"
    "- <suggestion 2>\n"
    "- <suggestion 3>\n\n"
    "Replace <rating> with a number between 0 and 1, and each <suggestion> with a brief recommendation."
)


message = {
    "role": "user",
    "content": [
        {"type": "image", "image": image},
        {"type": "text", "text": evaluation_instruction}
    ]
}

messages = [message]

text_prompt = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)

image_inputs, _ = process_vision_info(messages)

inputs = processor(
    text=[text_prompt],
    images=image_inputs,
    padding=True,
    return_tensors="pt",
)
inputs = inputs.to(model.device)

generated_ids = model.generate(**inputs, max_new_tokens=1300)

generated_ids_trimmed = [
    out_ids[len(inputs.input_ids[0]):] for out_ids in generated_ids
]
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)

evaluation_result = output_text[0]
print("=== Evaluation Result ===")
print(evaluation_result)

eval_file_path = os.path.join(eval_dir, "1.txt")
with open(eval_file_path, "w", encoding="utf-8") as f:
    f.write(evaluation_result)

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

=== Evaluation Result ===
### Ratings:
- Ergonomic & Functional Placements: 0.7
- Visual Harmony and Aesthetics: 0.6
- Proportionality and Scale: 0.8
- Readiness to Pay: 0.5

### Suggestions:
- Move the Wardrobe closer to the center of the room for better balance.
- Position the Desk Perturber near the Desk to facilitate interaction.
- Adjust the Coffee Table slightly further from the Chair for better space utilization.
- Reorient the Desk to align more closely with the North Wall for improved ergonomics.


In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3,4,5,6,7"


In [ ]:
import os
import torch
from PIL import Image
from transformers import Qwen2_5_VLForConditionalGeneration, AutoProcessor
from qwen_vl_utils import process_vision_info

image_dir = "/home/jovyan/shares/SR008.fs2/iana_kulichenko/Experiments/train_GRPO/images_2"
eval_dir = "/home/jovyan/shares/SR008.fs2/iana_kulichenko/Experiments/train_GRPO/evaluations"
os.makedirs(eval_dir, exist_ok=True)

image_files = [
    os.path.join(image_dir, f)
    for f in os.listdir(image_dir)
    if f.lower().endswith((".jpg", ".jpeg", ".png"))
]

def extract_number(filepath):
    base = os.path.splitext(os.path.basename(filepath))[0]
    return int(base)

sorted_files = sorted(image_files, key=extract_number)

model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2.5-VL-72B-Instruct",
    torch_dtype="auto",
    device_map="auto"
)
processor = AutoProcessor.from_pretrained("Qwen/Qwen2.5-VL-72B-Instruct", use_fast=False)

evaluation_instruction = (
    "As a professional interior designer, evaluate the following 2D layout visually. "
    "Provide exactly four ratings (each a number between 0 and 1) based on:\n\n"
    "- Ergonomic & Functional Placements: Comfort, accessibility, and practicality.\n"
    "- Visual Harmony and Aesthetics: Balance and overall appeal.\n"
    "- Proportionality and Scale: Correct sizes, rotations, and proportions.\n"
    "- Readiness to Pay: Value assessment (1 = willing to purchase, 0 = not).\n\n"
    "Also, list up to three one-sentence suggestions for coordinate adjustments under 'Suggestions'.\n\n"
    "Output exactly as:\n"
    "### Ratings:\n"
    "- Ergonomic & Functional Placements: <rating>\n"
    "- Visual Harmony and Aesthetics: <rating>\n"
    "- Proportionality and Scale: <rating>\n"
    "- Readiness to Pay: <rating>\n"
    "### Suggestions:\n"
    "- <suggestion 1>\n"
    "- <suggestion 2>\n"
    "- <suggestion 3>\n\n"
    "Replace <rating> with a number between 0 and 1, and each <suggestion> with a brief recommendation."
)

for img_path in sorted_files:
    print(f"Processing image: {img_path}")
    image = Image.open(img_path).convert("RGB")

    message = {
        "role": "user",
        "content": [
            {"type": "image", "image": image},
            {"type": "text", "text": evaluation_instruction}
        ]
    }
    messages = [message]

    text_prompt = processor.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )

    image_inputs, _ = process_vision_info(messages)
    inputs = processor(
        text=[text_prompt],
        images=image_inputs,
        padding=True,
        return_tensors="pt",
    )
    inputs = inputs.to(model.device)

    generated_ids = model.generate(**inputs, max_new_tokens=2048)
    generated_ids_trimmed = [
        out_ids[len(inputs.input_ids[0]):] for out_ids in generated_ids
    ]
    output_text = processor.batch_decode(
        generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
    )
    evaluation_result = output_text[0]

    base_name = os.path.splitext(os.path.basename(img_path))[0]
    eval_file_path = os.path.join(eval_dir, f"{base_name}_evaluation.txt")
    with open(eval_file_path, "w", encoding="utf-8") as f:
        f.write(evaluation_result)

Loading checkpoint shards:   0%|          | 0/38 [00:00<?, ?it/s]

Processing image: /home/jovyan/shares/SR008.fs2/iana_kulichenko/Experiments/train_GRPO/images_2/1.png
Processing image: /home/jovyan/shares/SR008.fs2/iana_kulichenko/Experiments/train_GRPO/images_2/3.png
Processing image: /home/jovyan/shares/SR008.fs2/iana_kulichenko/Experiments/train_GRPO/images_2/4.png
Processing image: /home/jovyan/shares/SR008.fs2/iana_kulichenko/Experiments/train_GRPO/images_2/5.png
Processing image: /home/jovyan/shares/SR008.fs2/iana_kulichenko/Experiments/train_GRPO/images_2/6.png
Processing image: /home/jovyan/shares/SR008.fs2/iana_kulichenko/Experiments/train_GRPO/images_2/8.png
Processing image: /home/jovyan/shares/SR008.fs2/iana_kulichenko/Experiments/train_GRPO/images_2/9.png
Processing image: /home/jovyan/shares/SR008.fs2/iana_kulichenko/Experiments/train_GRPO/images_2/10.png
Processing image: /home/jovyan/shares/SR008.fs2/iana_kulichenko/Experiments/train_GRPO/images_2/11.png
Processing image: /home/jovyan/shares/SR008.fs2/iana_kulichenko/Experiments/trai

In [3]:
import os
import torch
from PIL import Image
from transformers import Qwen2_5_VLForConditionalGeneration, AutoProcessor
from qwen_vl_utils import process_vision_info

image_dir = "/home/jovyan/shares/SR008.fs2/iana_kulichenko/Experiments/train_GRPO/images_2"
eval_dir = "/home/jovyan/shares/SR008.fs2/iana_kulichenko/Experiments/train_GRPO/evaluations_2"
os.makedirs(eval_dir, exist_ok=True)

image_files = [
    os.path.join(image_dir, f)
    for f in os.listdir(image_dir)
    if f.lower().endswith((".jpg", ".jpeg", ".png"))
]

def extract_number(filepath):
    base = os.path.splitext(os.path.basename(filepath))[0]
    return int(base)

sorted_files = sorted(image_files, key=extract_number)

model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2.5-VL-72B-Instruct",
    torch_dtype="auto",
    device_map="auto"
)
processor = AutoProcessor.from_pretrained("Qwen/Qwen2.5-VL-72B-Instruct", use_fast=False)

evaluation_instruction = (
    "As a professional designer with expertise in spatial layout and interior design, "
    "please provide a comprehensive evaluation of the following 2D layout. \n"
    " Make detailed analysis, suggest in detailes how to improve and than provide answer.\n "
    "Assign exactly six ratings (each a number from 0 to 1) based on the following criteria:\n\n"
    "- All Objects Within Room Bounds: Determine whether every object is placed fully within the room boundaries.\n"
    "- No Objects Overlap: Assess whether the objects are well-spaced and not overlap.\n"
    "- Overall Layout Quality: Evaluate the visual harmony, balance, and overall effectiveness of the design.\n"
    "- Functionality of the Layout: Assess how well the layout meets practical requirements and optimizes space usage.\n"
    "- Ergonomic Placements: Judge the positioning of elements in terms of comfort, accessibility, and user experience.\n"
    "- Readiness to Pay: Indicate whether you would consider this design valuable enough to purchase (use 1 for yes, 0 for no).\n"
    "Please ensure that the output is formatted exactly as follows:\n"
    "### Ratings:\n"
    "- All Objects Within Room Bounds: <rating>\n"
    "- Objects Not Overlap: <rating>\n"
    "- Overall Layout Quality: <rating>\n"
    "- Functionality of the Layout: <rating>\n"
    "- Ergonomic Placements: <rating>\n"
    "- Readiness to Pay: <rating>\n"
    "Replace <rating> with a number between 0 and 1."
)

for img_path in sorted_files:
    print(f"Processing image: {img_path}")
    image = Image.open(img_path).convert("RGB")

    message = {
        "role": "user",
        "content": [
            {"type": "image", "image": image},
            {"type": "text", "text": evaluation_instruction}
        ]
    }
    messages = [message]

    text_prompt = processor.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )

    image_inputs, _ = process_vision_info(messages)
    inputs = processor(
        text=[text_prompt],
        images=image_inputs,
        padding=True,
        return_tensors="pt",
    )
    inputs = inputs.to(model.device)

    generated_ids = model.generate(**inputs, max_new_tokens=2048)
    generated_ids_trimmed = [
        out_ids[len(inputs.input_ids[0]):] for out_ids in generated_ids
    ]
    output_text = processor.batch_decode(
        generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
    )
    evaluation_result = output_text[0]

    base_name = os.path.splitext(os.path.basename(img_path))[0]
    eval_file_path = os.path.join(eval_dir, f"{base_name}_evaluation.txt")
    with open(eval_file_path, "w", encoding="utf-8") as f:
        f.write(evaluation_result)

Loading checkpoint shards:   0%|          | 0/38 [00:00<?, ?it/s]

Processing image: /home/jovyan/shares/SR008.fs2/iana_kulichenko/Experiments/train_GRPO/images_2/1.png
Processing image: /home/jovyan/shares/SR008.fs2/iana_kulichenko/Experiments/train_GRPO/images_2/3.png
Processing image: /home/jovyan/shares/SR008.fs2/iana_kulichenko/Experiments/train_GRPO/images_2/4.png
Processing image: /home/jovyan/shares/SR008.fs2/iana_kulichenko/Experiments/train_GRPO/images_2/5.png
Processing image: /home/jovyan/shares/SR008.fs2/iana_kulichenko/Experiments/train_GRPO/images_2/6.png
Processing image: /home/jovyan/shares/SR008.fs2/iana_kulichenko/Experiments/train_GRPO/images_2/8.png
Processing image: /home/jovyan/shares/SR008.fs2/iana_kulichenko/Experiments/train_GRPO/images_2/9.png
Processing image: /home/jovyan/shares/SR008.fs2/iana_kulichenko/Experiments/train_GRPO/images_2/10.png
Processing image: /home/jovyan/shares/SR008.fs2/iana_kulichenko/Experiments/train_GRPO/images_2/11.png
Processing image: /home/jovyan/shares/SR008.fs2/iana_kulichenko/Experiments/trai

In [4]:
import os
import torch
from PIL import Image
from transformers import Qwen2_5_VLForConditionalGeneration, AutoProcessor
from qwen_vl_utils import process_vision_info

image_dir = "/home/jovyan/shares/SR008.fs2/iana_kulichenko/Experiments/train_GRPO/images_2"
eval_dir = "/home/jovyan/shares/SR008.fs2/iana_kulichenko/Experiments/train_GRPO/evaluations"
os.makedirs(eval_dir, exist_ok=True)

image_files = [
    os.path.join(image_dir, f)
    for f in os.listdir(image_dir)
    if f.lower().endswith((".jpg", ".jpeg", ".png"))
]
if not image_files:
    raise ValueError("No image files found in the specified folder.")
img_path = image_files[0]

image = Image.open(img_path).convert("RGB")
model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2.5-VL-72B-Instruct",
    torch_dtype="auto",
    device_map="auto"
)
processor = AutoProcessor.from_pretrained("Qwen/Qwen2.5-VL-72B-Instruct", use_fast=False)
evaluation_instruction = (
    "As a professional designer with expertise in spatial layout and interior design, "
    "please provide a comprehensive evaluation of the following 2D layout. \n"
    " Make detailed analysis, suggest in detailes how to improve and than provide answer.\n "
    "Assign exactly six ratings (each a number from 0 to 1) based on the following criteria:\n\n"
    "- All Objects Within Room Bounds: Determine whether every object is placed fully within the room boundaries.\n"
    "- No Objects Overlap: Assess whether the objects are well-spaced and not overlap.\n"
    "- Overall Layout Quality: Evaluate the visual harmony, balance, and overall effectiveness of the design.\n"
    "- Functionality of the Layout: Assess how well the layout meets practical requirements and optimizes space usage.\n"
    "- Ergonomic Placements: Judge the positioning of elements in terms of comfort, accessibility, and user experience.\n"
    "- Readiness to Pay: Indicate whether you would consider this design valuable enough to purchase (use 1 for yes, 0 for no).\n"
    "Please ensure that the output is formatted exactly as follows:\n"
    "### Ratings:\n"
    "- All Objects Within Room Bounds: <rating>\n"
    "- Objects Not Overlap: <rating>\n"
    "- Overall Layout Quality: <rating>\n"
    "- Functionality of the Layout: <rating>\n"
    "- Ergonomic Placements: <rating>\n"
    "- Readiness to Pay: <rating>\n"
    "Replace <rating> with a number between 0 and 1."
)

message = {
    "role": "user",
    "content": [
        {"type": "image", "image": image},
        {"type": "text", "text": evaluation_instruction}
    ]
}

messages = [message]

text_prompt = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)

image_inputs, _ = process_vision_info(messages)

inputs = processor(
    text=[text_prompt],
    images=image_inputs,
    padding=True,
    return_tensors="pt",
)
inputs = inputs.to(model.device)

generated_ids = model.generate(**inputs, max_new_tokens=2048)

generated_ids_trimmed = [
    out_ids[len(inputs.input_ids[0]):] for out_ids in generated_ids
]
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)

evaluation_result = output_text[0]
print("=== Evaluation Result ===")
print(evaluation_result)

Loading checkpoint shards:   0%|          | 0/38 [00:00<?, ?it/s]

=== Evaluation Result ===
### Evaluation

#### Analysis:
The provided 2D layout represents a meeting hall measuring 8.5 meters by 4.0 meters. The layout includes five objects: a desk perturber, a wardrobe, a chair, a desk, and a coffee table. Here's a detailed analysis:

1. **All Objects Within Room Bounds**: All objects appear to be placed within the room boundaries, as none of them extend beyond the dashed lines indicating the walls.

2. **No Objects Overlap**: The objects do not overlap with each other, ensuring clear pathways and individual space for each item.

3. **Overall Layout Quality**: The current layout lacks visual harmony and balance. The objects are scattered without a clear focal point or logical arrangement. For example, the wardrobe is placed near the center of the room, which is unusual for such an item. The chair is isolated near the north wall, and the desk is placed far from it, which may not be ideal for a meeting setup.

4. **Functionality of the Layout**: The f